<a href="https://colab.research.google.com/github/hopecris31/Reddit_Transformer/blob/master/Final_Project_with_Trained_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers[sentencepiece]
!pip install datasets
!pip install evaluate
!pip install google

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import transformers
import math
import torch
from google.colab import drive
from datasets import load_dataset, load_from_disk
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_metric
import numpy as np

drive.mount('/content/drive')

checkpoint = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
model_path = '/content/drive/MyDrive/NLP/ChatGRT'

model = AutoModelForCausalLM.from_pretrained(model_path)

In [5]:
from datasets import load_from_disk

eval_dataset_path = "/content/drive/MyDrive/NLP/eval_dataset"
eval_dataset = load_from_disk(eval_dataset_path)


In [6]:
def tokenize_function(example):
    tokenizer.pad_token = tokenizer.eos_token
    return tokenizer(example["subreddit"], example["content"], padding="max_length", truncation=True, max_length=32)

tokenized_datasets = eval_dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=eval_dataset.column_names)

In [7]:
block_size = tokenizer.model_max_length

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum([ex for ex in examples[k] if isinstance(ex, list)], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [8]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [9]:
training_args = TrainingArguments(
    output_dir="output",
    per_device_eval_batch_size=1,  # Set evaluation batch size to 1
    eval_steps=1,
    eval_accumulation_steps=10,
    do_eval=True

)

In [10]:
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import sacrebleu

In [12]:

def compute_metrics(eval_preds):
    accuracy_metric = load_metric("accuracy")
    precision_metric = load_metric("precision")
    recall_metric = load_metric("recall")
    f1_metric = load_metric("f1")

    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels)
    recall = recall_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)

    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    bleu = sacrebleu.corpus_bleu(decoded_predictions, [decoded_labels]).score

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'bleu': bleu
    }


In [13]:
trainer = Trainer(
    model=AutoModelForCausalLM.from_pretrained(model_path),
    args=training_args,
    train_dataset=lm_datasets,
    eval_dataset=lm_datasets,
    compute_metrics=compute_metrics,
)

In [ ]:
eval_results = trainer.evaluate()

In [ ]:
print(f"Perplexity: {eval_results['eval_loss']:.2f}")

In [ ]:
print(f"Accuracy: {eval_results['accuracy']:.2f}")
print(f"Precision: {eval_results['precision']:.2f}")
print(f"Recall: {eval_results['recall']:.2f}")
print(f"F1 Score: {eval_results['f1']:.2f}")

In [ ]:
save_path = '/content/drive/MyDrive/NLP/evaluation_results.txt'

with open(save_path, 'w') as f:
    f.write(f"Accuracy: {eval_results['accuracy']:.2f}\n")
    f.write(f"Precision: {eval_results['precision']:.2f}\n")
    f.write(f"Recall: {eval_results['recall']:.2f}\n")
    f.write(f"F1 Score: {eval_results['f1']:.2f}\n")

In [ ]:
save_path_bleu = '/content/drive/MyDrive/BLEU.txt'

with open(save_pat_bleuh, 'w') as f:
    f.write(f'BLEU score: {bleu.score}\n')

In [ ]:
for batch in eval_dataloader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

    total_loss += loss.item()
    total_eval_steps += 1

eval_loss = total_loss / total_eval_steps
perplexity = math.exp(eval_loss)
print(f"Perplexity: {perplexity:.2f}")

In [ ]:
#from datasets import load_dataset

In [ ]:
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import sacrebleu

eval_dataloader = DataLoader(lm_datasets, batch_size=1)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

decoded_predictions = []
decoded_labels = []

with torch.no_grad():
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        
        outputs = model.generate(input_ids, attention_mask=attention_mask)
        predictions = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        decoded_predictions.extend(predictions)
        
        label_strings = tokenizer.batch_decode(labels, skip_special_tokens=True)
        decoded_labels.extend(label_strings)

bleu = sacrebleu.corpus_bleu(decoded_predictions, [decoded_labels]).score
print("BLEU score:", bleu)


In [ ]:
#raw_datasets = load_dataset("reddit", split="train[:5%]")
#raw_datasets

In [ ]:
#
#tokenized_datasets

In [ ]:

#tokenized_datasets[1]


In [ ]:
#block_size = tokenizer.model_max_length

"""
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum([ex for ex in examples[k] if isinstance(ex, list)], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result
"""

In [ ]:
# lm_datasets = tokenized_datasets.map(
#     group_texts,
#     batched=True,
#     batch_size=1000,
#     num_proc=4,
# )

In [ ]:
# lm_datasets

In [ ]:
# tokenizer.decode(lm_datasets[1]["input_ids"])

In [ ]:
# from transformers import TrainingArguments, Trainer

In [ ]:
# model_name = checkpoint.split("/")[-1]
# training_args = TrainingArguments(
#     f"{model_name}-finetuned-reddit",
#     evaluation_strategy = "epoch",
#     learning_rate=2e-5,
#     weight_decay=0.01,
#     push_to_hub=False
# )

In [ ]:
# from google.colab import drive


In [ ]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=lm_datasets,
#     eval_dataset=lm_datasets,
# )

In [ ]:
# trainer.train()

In [ ]:
# drive.mount('/content/drive')
# PATH = '/content/drive/MyDrive/NLP' # [CHANGE ME]
# #'/content/drive/MyDrive/NLP/ChatGR'
# trainer.save_model (PATH + "/ChatGRT")

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:

sample_1_p = tokenized_datasets[300]
#print(sample_1_p['subreddit'], '\n', sample_1_p['content'])

user_input_subreddit = input("Enter Subreddit: ")
user_input_prompt = input("Enter Prompt: ")


input_p = tokenizer(user_input_subreddit, 
                    user_input_prompt, 
                    padding=True,
                    truncation=True, return_tensors='pt')
input_p = input_p.to('cuda')

model(**input_p)